$\textbf{Problem 4.} \hspace{2mm}$ Consider the data in Table 5.7 of the textbook. Use TEST as $X$,  RACE as $Z$, and JPERF as response variable $Y$. $X$ is quantitative predictor variable, $Z$ is a binary indicator variable. Instead of using all 20 data points, you would use 18 datapoints including the first 9 data points from each group.

$\textbf{Note:} \hspace{2mm}$ We use all 20 data points in the solution. If you used 18 points, your results should be very similar to our results, you can still compare them.

In [17]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

data = pd.read_csv('preemployment.txt', sep='\t') 

X = data['TEST']  # predictor variable
Y = data['JPERF'] # response variable
Z = data['RACE']  # indicator variable

$\textbf{1.} \hspace{2mm}$ In order to answer if the minority group $(Z= 1)$ has the same coefficients as the non-minority group $(Z= 0)$, first provide the pooled model, and separate model for each group, and then perform a F-test to draw your conclusion? Can you use a t-test to draw the conclusion?

$\textbf{Answer:} \hspace{2mm}$ A pooled model refers to a statistical analysis where data from multiple groups are combined (pooled) and treated as a single dataset for analysis. Why do we do that?

If there is a common underlying relationship, or the differences between groups are not of primary interest, pooling the data can increase the sample size, leading to increased statistical power. 

In such case, we do not need to fit two regression lines on two datasets (or might be indicated by a 0-1 variable $Z$ in one dataset), instead we fit only one line, which also makes the analysis simpler. To do this, we will need a hypothesis testing to understand the relationship.

In our question, we first ignore the difference $Z=0$ or $Z=1$, which is a race distinction in society for our case, and consider $X$ as the predictor variable and $Y$ as the response variable of whole data. The pooled model is

$$y_{i}= \beta^{pool}_0 + \beta^{pool}_1 x_i + \epsilon^{pool}_{i}.$$

For minority rows of the data, the minority model is

$$y^{m}_{i} = \beta^{m}_{0} + \beta^{m}_{1} x^{m}_{i} + \epsilon^{m}_{i},$$

and for non-minority rows, the non-minority model is

$$y^{n}_{i} = \beta^{n}_{0} + \beta^{n}_{1} x_{i} + \epsilon^{n}_{i}.$$


We start with fitting the regression line of the pooled model. Note that the number predictors in the data is $p=1$. 

In [26]:
n = len(Y)
p = 1

X_design_pooled = np.column_stack((np.ones(n),X))
H_pooled = X_design_pooled @ np.linalg.pinv(X_design_pooled.T @ X_design_pooled)@ X_design_pooled.T

Y_hat_pooled = H_pooled @ Y

residuals_pooled = Y-Y_hat_pooled

Beta_hat_pooled = np.linalg.pinv(X_design_pooled.T @ X_design_pooled)@ X_design_pooled.T@ Y

Let's seperate the dataset according to the difference, $Z=1$ or $Z = 0$.

In [ ]:
X_minority = X[Z == 1]    
Y_minority = Y[Z == 1]

X_nonminority = X[Z == 0]
Y_nonminority = Y[Z == 0]

We fit the regression lines of minority and non-minority models.

In [23]:
# Regression line for the minority group

n = len(Y_minority)

X_design_minority = np.column_stack((np.ones(n),X_minority))
H_minority = X_design_minority @ np.linalg.pinv(X_design_minority.T @ X_design_minority)@ X_design_minority.T

Y_hat_minority = H_minority @ Y_minority
residuals_minority = Y_minority - Y_hat_minority
Beta_hat_minority = np.linalg.pinv(X_design_minority.T @ X_design_minority)@ X_design_minority.T @ Y_minority.T


# Regression line for the non-minority group

X_design_nonminority = np.column_stack((np.ones(n),X_nonminority))
H_nonminority = X_design_nonminority @ np.linalg.pinv(X_design_nonminority.T @ X_design_nonminority)@ X_design_nonminority.T

Y_hat_nonminority = H_nonminority @ Y_nonminority
residuals_nonminority = Y_nonminority - Y_hat_nonminority
Beta_hat_nonminority = np.linalg.pinv(X_design_nonminority.T @ X_design_nonminority)@ X_design_nonminority.T @ Y_nonminority.T


See the cofficient vectors of pooled, minoirty and non-minority models.

In [25]:
print('Pooled model:', Beta_hat_pooled)
print('Minority model:', Beta_hat_minority)
print('Non-minority model:', Beta_hat_nonminority)

Pooled model: [1.03497323 2.36053467]
Minority model: [0.09711523 3.31094801]
Non-minority model: [2.01028229 1.31340219]


Now, we need the null hypothesis 

$$H_0 : \beta^{m}_{1} = \beta^{n}_{1}, \hspace{3mm} \beta^{m}_{0} = \beta^{n}_{0}$$ 

against the alternative that there are substantial differences in these parameters. How do we do such a test? This is explained on Page 140 in the textbook. Let's discuss it here in detail. 

We will find an equivalent hypothesis to $H_0$. Remind that models are 

$\textbf{Pooled model:}$ $$y_{i}= \beta^{pool}_0 + \beta^{pool}_1 x_i + \epsilon^{pool}_{i}.$$

$\textbf{Minority model:}$ $$y^{m}_{i} = \beta^{m}_{0} + \beta^{m}_{1} x^{m}_{i} + \epsilon^{m}_{i},$$

$\textbf{Non-minority model:}$ $$y^{n}_{i} = \beta^{n}_{0} + \beta^{n}_{1} x_{i} + \epsilon^{n}_{i}.$$

Consider the following revised model that contains the term of interaction $ z_i x^{pool}_{i}$ between the indicator of groups and data points.  

$\textbf{Revised model:}$ $$y_{i} = \beta^{pool}_{0} + \beta^{pool}_1 x^{pool}_{i} + \gamma z_{i} + \delta (z_i x^{pool}_{i})+ \epsilon^{pool}_i,$$

where $\hspace{1mm} \delta = \beta^{m}_{1} - \beta^{pool}_1$ and $\gamma = \beta^m_{0} - \beta^{pool}_0$.

$\textbf{Claim:} \hspace{1mm}$ Revised model is equivalent to minority and non-minority models.

$\textbf{Proof:} \hspace{1mm}$ If $x^{pool}_{i}$ is a non-minority point, then $z_i = 0$ and the revised model becomes the pooled model. If $x^{pool}_{i}$ is a minority point, substituting $\delta = \beta^{m}_{1} - \beta^{pool}_1$, $\gamma = \beta^m_{0} - \beta^{pool}_0$ and $z_i = 1$ in the revised model, we again obtain the pooled model. Therefore, the comparison between pooled model and revised model is equivalent to a comparison between pooled model and minority (or non-minority) model. This provides the equivalency. $\square$

This equivalency is important because the datasets of revised and pooled models are the same unlike other models.

Revised model can be viewed as a full model $(FM)$ and pooled model as reduced model $(RM)$. Because, pooled model is obtained from non-minority model. This can be seen by setting $\gamma = \delta =0$ in the revised model.

Therefore, the null hypothesis 

$$H_0 : \beta^{m}_{1} = \beta^{n}_{1}, \hspace{3mm} \beta^{m}_{0} = \beta^{n}_{0}$$ 

is equivalent to the null hypothesis $H_{0}: \gamma = \delta = 0$. 

The number of predictors in the reduced (pooled) model is 1, and so $df(RM) = n-1-1=18$.

There are 3 predictors in the full (revised) model. Because, $x^{pool}_{i}$, $\gamma z_{i}$ and $z_i x^{pool}_{i}$ are different variables. So, $df(FM) = n-3-1=16$.

 From the general formula of F-statistic, we obtain

$$F^\star = \dfrac{SSE(RM) -SSE(FM)}{2} \bigg/ \dfrac{SSE(FM)}{16} \thicksim F(2, 16).$$

To calculate $SSE(FM)$, we need to fit the regression line of revised model.

In [28]:
X_pooled_times_Z = np.multiply(X, Z)

X_revised_design = np.column_stack((np.ones(n), X, Z, X_pooled_times_Z))
H_revised = X_revised_design @ np.linalg.pinv(X_revised_design.T @ X_revised_design) @ X_revised_design.T 

Y_hat_revised = H_revised @ Y

beta_hat_revised = (np.linalg.pinv(X_revised_design.T @ X_revised_design) @ X_revised_design.T) @ Y_hat_revised
beta_hat_revised

array([ 2.01028229,  1.31340219, -1.91316707,  1.99754582])

Let's calculate $SSE(RM)$ and $SSE(FM)$ and test the null hypothesis.

In [31]:
import scipy.stats


# Reduced model (pooled)
SSE_RM = np.sum(np.multiply(residuals_pooled, residuals_pooled))

# Full Model (Revised)
residuals_revised = Y - Y_hat_revised
SSE_FM = np.sum(np.multiply(residuals_revised, residuals_revised))


F_statistic = ((SSE_RM - SSE_FM) /2) / ((SSE_FM / 16))

alpha = 0.05

p_value = scipy.stats.f.cdf(F_statistic, 2, 16)

print("SSE(RM):", SSE_RM)
print("SSE(FM):", SSE_FM)
print("F_statistic:", F_statistic)
print("p-value:", p_value)
print("alpha:", alpha)

if (p_value < alpha):
    print('Null hypothesis is rejected.')
else:
    print('failed to reject the null hypothesis.')

SSE(RM): 45.5682968283022
SSE(FM): 31.655473162338446
F_statistic: 3.5160614645346824
p-value: 0.9457639437352865
alpha: 0.05
failed to reject the null hypothesis.


The null hypothosis $H_{0}: \gamma = \delta = 0$ is rejected. 

Therefore, $$H_0 : \beta^{m}_{1} = \beta^{n}_{1}, \hspace{3mm} \beta^{m}_{0} = \beta^{n}_{0}$$ is also rejected.

Cofficients of minority and non-minority models are not the same. 

To be able to use t-test, we need to test only one cofficient. So, $H_0: \gamma=0$ or  $H_0:\delta= 0$ must be equivalent to 

$H_{0}: \gamma = \delta = 0$, which is not always true.

$ \textbf{Part 2.}: \hspace{2mm}$ If we know as a fact that the two groups have the same intercept, we are interested in testing if the slope of $X$ is also the same. What would you do? Would you use a t-test or F-test?

$ \textbf{Answer}: \hspace{2mm}$ Assume that $\beta^{m}_{0}=\beta^{n}_{0}$. In that case, we have

Pooled model: $$\hspace{2mm} y^{pool}_{i} = \beta^{pool}_{0} + \beta^{pool}_{1} x^{pool}_{i} + \epsilon^{pool}_{i},$$

Minority model: $$\hspace{2mm} y^{m}_{i} = \beta^{pool}_{0} + \beta^{m}_{1} x^{m}_{i} + \epsilon^{m}_{i},$$

Non-minority model: $$\hspace{2mm} y^{n}_{i} = \beta^{pool}_{0} + \beta^{n}_{1} x_{i} + \epsilon^{n}_{i}$$

and $\gamma = \beta^{m}_0 -\beta^{pool}_{0} = 0$. Then the revised model becomes

$$y_{i} = \beta^{pool}_{0} + \beta^{pool}_1 x^{pool}_{i} + \delta (z_i x^{pool}_{i})+ \epsilon^{pool}_i.$$

It is now enough to test the hypothesis $H_0: \delta=0$ to show the equivalency that we discussed above. Hence, we can use 

t-test.

